[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milioe/Intro-APIs-Python-ML/blob/main/1_Consumir_APIs.ipynb)

# 🚀 Clase 1: Consumir APIs

**APIs de Python: de cero a ML**

---

## 📋 ¿Qué aprenderás hoy?

1. ¿Qué es una API y por qué nos importa?
2. Consumir la API de INEGI (datos económicos de México)
3. Usar SDKs como `yfinance` para datos financieros
4. Conectar con Gemini (IA) para analizar datos

---


## 🛠️ Instalación de dependencias

Ejecuta esta celda para instalar las librerías que usaremos (Tienes que descomentarla)

In [ ]:
# %pip install requests pandas matplotlib yfinance google-generativeai -q

In [ ]:
# Importamos las librerías
import requests
import pandas as pd
import matplotlib.pyplot as plt

print("✅ ¡Librerías cargadas correctamente!")


---

# 🍽️ ¿Qué es una API?

## La Analogía del Restaurante

Imagina que estás en un restaurante:

| Restaurante | Mundo de las APIs |
|-------------|-------------------|
| Tú (cliente) | Tu programa/código |
| Mesero | **API** |
| Cocina | Servidor/Base de datos |
| Menú | Documentación de la API |
| Tu orden | **Request** (petición) |
| Tu platillo | **Response** (respuesta) |

**API = Application Programming Interface** (Interfaz de Programación de Aplicaciones)

> 💡 Una API es un **intermediario** que permite que dos sistemas se comuniquen sin conocer los detalles internos del otro.

---

## ¿Por qué nos importa como economistas?

- 📊 **Acceso a datos en tiempo real**: INEGI, Banxico, World Bank, entre otras fuentes
- 🤖 **Automatización**: No más descargar CSVs manualmente
- 🔄 **Reproducibilidad**: Tu análisis siempre usa datos actualizados
- 🚀 **Escala**: Procesar miles de consultas automáticamente


---

# 🇲🇽 Tu Primera API: INEGI

Vamos directo con datos económicos reales de México.

**INEGI** (Instituto Nacional de Estadística y Geografía) tiene una API pública para acceder a indicadores económicos.

📚 [Documentación oficial](https://www.inegi.org.mx/servicios/api_indicadores.html)

## ¿Cómo funciona una llamada a API?

```
https://www.inegi.org.mx/app/api/indicadores/.../INDICATOR/{indicador}/.../BIE/2.0/{token}?type=json
```

| Elemento | Qué es | Ejemplo |
|----------|--------|---------|
| **URL** | Dirección de la API | `https://www.inegi.org.mx/app/api/...` |
| **Parámetros** | Datos que enviamos | `indicador`, `token` |
| **Response** | Lo que nos devuelve | JSON con datos |

| Código de respuesta | Significado |
|---------------------|-------------|
| 200 | ✅ Éxito |
| 401 | 🔐 No autorizado (falta API key) |
| 404 | 🔍 No encontrado |
| 500 | 💥 Error del servidor |


In [ ]:
# 1. Declaramos el TOKEN (cambia este token por el tuyo, recuerda que es privado y se declara en mayúsculas)
TOKEN_INEGI = ""

# 2. Declaramos el INDICADOR que queremos consultar (628194 = Inflación INPC)
indicador_inflacion = "6207078924"

# 3. Construimos la URL completa con nuestros parámetros
url_inegi = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/496150/es/00/false/BISE/2.0/{TOKEN_INEGI}?type=json"

In [ ]:
# 4. Hacemos la PETICIÓN (request) a la API
response = requests.get(url_inegi)

print(f"✅ Status: {response.status_code}")  # 200 = éxito

In [ ]:
# 5. Exploramos qué hay dentro de la respuesta
datos_inegi = response.json()

# Opcional: Imprimimos el JSON completo (nos saldrán muchos registros)
# print(datos_inegi)

# Navegamos hasta las observaciones: datos_inegi -> 'Series' -> [0] -> 'OBSERVATIONS'
serie = datos_inegi['Series'][0]['OBSERVATIONS']

print("🔍 Primeros 5 registros:")
serie[:5]


In [ ]:
# 6. Convertimos a DataFrame de pandas y limpiamos
df_igae = pd.DataFrame(serie)
df_igae['TIME_PERIOD'] = pd.to_datetime(df_igae['TIME_PERIOD'], format='%Y/%m')
df_igae['OBS_VALUE'] = pd.to_numeric(df_igae['OBS_VALUE'])
df_igae = df_igae.rename(columns={'TIME_PERIOD': 'Fecha', 'OBS_VALUE': 'IGAE'})

# Observamos los primeros registros
df_igae.head(10)


In [ ]:
# 7. Graficamos el IGAE
plt.figure(figsize=(12, 5)) # Ajustamos el tamaño de la gráfica
plt.plot(df_igae['Fecha'], df_igae['IGAE'], color='#006847', linewidth=2) # Graficamos la serie
plt.title('IGAE - Indicador Global de la Actividad Económica', fontsize=14, fontweight='bold') # Declaramos el título
plt.xlabel('Fecha') # Etiqueta del eje x
plt.ylabel('Índice') # Etiqueta del eje y
plt.grid(True, alpha=0.3) # Agregamos una cuadrícula
plt.tight_layout() # Ajustamos el layout para que no se superpongan las etiquetas
plt.show() # Mostramos la gráfica


### 🎯 Tu turno: Elige un indicador

Escoge el indicador que más te interese e imprime los datos


In [ ]:
# 🎯 EJERCICIO: Cambia el indicador para obtener el PIB
indicador = "______"  # <- Pon aquí el código del PIB

# Lo obtienes de https://www.inegi.org.mx/servicios/api_indicadores.html en donde dice "El token puede obtenerse al registrarse aquí"
TOKEN_INEGI = ""

# Copia y pega la URL que te de INEGI y añade tu token
url = f""

# Hacemos la petición a la API
response = requests.get(url)

# Exploramos qué hay dentro de la respuesta
datos = response.json()

# Extrae los datos y crea un DataFrame
serie = datos['Series'][0]['OBSERVATIONS']
df = pd.DataFrame(serie)

print("Nombre del indicador:", datos['Series'][0]['METADATA'][0]['VALUE'])
df.tail()

---

# 📦 SDKs: El Atajo de los Programadores

## ¿Qué es un SDK?

**SDK = Software Development Kit**

Un SDK es una librería que "envuelve" una API para hacerla más fácil de usar.

| Sin SDK (API cruda) | Con SDK |
|---------------------|----------|
| Construir URLs manualmente | Funciones simples |
| Manejar autenticación | Se encarga solo |
| Parsear JSON | Devuelve DataFrames |
| ~15 líneas de código | ~2 líneas de código |

---

## 📈 yfinance: Datos Financieros en 2 Líneas

`yfinance` es un SDK que conecta con Yahoo Finance para obtener datos de:
- Acciones (AAPL, MSFT, AMZN...)
- Índices (^GSPC, ^MXX...)
- Divisas (USDMXN=X, EURUSD=X...)
- Criptomonedas (BTC-USD, ETH-USD...)


In [ ]:
import yfinance as yf

# Acción de Nvidia
ticker = yf.Ticker("NVDA")

# Obtenemos datos históricos (últimos 2 años)
df_nvda = ticker.history(period="2y")

df_nvda.head()


In [ ]:
# ¡Graficamos!
plt.figure(figsize=(12, 5))
plt.plot(df_nvda.index, df_nvda['Close'], color='#76B900', linewidth=2)  # Verde Nvidia
plt.title('NVIDIA (NVDA) - Precio de la acción', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Precio (USD)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


### 🎯 Tu turno: Elige un ticker y grafícalo

| Activo | Ticker |
|--------|--------|
| Apple | AAPL |
| Microsoft | MSFT |
| Tesla | TSLA |
| Amazon | AMZN |
| Bitcoin | BTC-USD |
| Oro | GC=F |
| S&P 500 | ^GSPC |
| IPC México | ^MXX |


In [ ]:
# 🎯 EJERCICIO: Elige un ticker y grafícalo

ticker = "______"  # <- Pon aquí tu ticker

# Obtenemos los datos
df = yf.Ticker(ticker).history(period="1y")

# Graficamos
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['Close'], linewidth=2)
plt.title(f'{ticker} - Precio del último año', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


---

# 🏦 API de Banxico

El Banco de México tiene una API para acceder a indicadores económicos: tipo de cambio, tasas de interés, inflación, etc.

📚 [Documentación oficial](https://www.banxico.org.mx/SieAPIRest/service/v1/)

Para obtener tu token: [Registro SIE API](https://www.banxico.org.mx/SieAPIRest/service/v1/token)

Consulta el catálogo de series [aquí](https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries)


In [ ]:
# 1. Token de Banxico (regístrate para obtener el tuyo)
TOKEN_BANXICO = ""

# 2. Construimos la URL
url_banxico = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF4/datos"

# 3. Headers con el token
headers = {"Bmx-Token": TOKEN_BANXICO}

# 4. Hacemos la petición
response = requests.get(url_banxico, headers=headers)
print(f"Status: {response.status_code}")

In [ ]:
# 6. Exploramos la respuesta
datos_banxico = response.json()

# Navegamos hasta los datos
serie = datos_banxico['bmx']['series'][0]['datos']

print(f"📊 Encontramos {len(serie)} registros\n")
print("🔍 Primeros 5:")
serie[:5]


In [ ]:
# 7. Convertimos a DataFrame y graficamos
df_banxico = pd.DataFrame(serie)

# Limpiamos los datos
df_banxico['fecha'] = pd.to_datetime(df_banxico['fecha'], format='%d/%m/%Y')
df_banxico['dato'] = df_banxico['dato'].str.replace(',', '').astype(float)

# Graficamos
plt.figure(figsize=(12, 5))
plt.plot(df_banxico['fecha'], df_banxico['dato'], linewidth=1.5, color='#006847')
plt.title('Billetes y monedas en circulación (Banxico)', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Miles de Pesos')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


---

# 🤖 Gemini: IA para Analizar Datos

**Gemini** es el modelo de IA de Google. Tiene una API gratuita que podemos usar para:
- Interpretar datos
- Generar resúmenes
- Responder preguntas sobre economía

## Obtener tu API Key (gratis)

1. Ve a [Google AI Studio](https://aistudio.google.com/app/apikey)
2. Crea una API Key
3. Cópiala y pégala abajo


In [ ]:
import google.generativeai as genai

# 🔑 Pon tu API key aquí
GEMINI_API_KEY = ""

# Configuramos Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Creamos el modelo
modelo = genai.GenerativeModel('gemini-2.0-flash')

print("✅ Gemini configurado correctamente")


In [ ]:
# Probamos con una pregunta simple
respuesta = modelo.generate_content("¿Qué es la inflación? Explícalo en 2 oraciones.")

print(respuesta.text)


## 🔥 Combinando APIs: INEGI + Gemini

Ahora viene lo interesante: vamos a pasarle datos de INEGI a Gemini para que los analice.


In [ ]:
# Tomamos las últimas 25 observaciones del IGAE (solo Fecha e IGAE)
ultimos_datos = df_igae[['Fecha', 'IGAE']].tail(25).to_string()

# Creamos el prompt
prompt = f"""
Eres un economista experto en México. Analiza los siguientes datos del IGAE 
(Indicador Global de la Actividad Económica) y da un breve análisis de la tendencia. 
Sé conciso (máximo 3 párrafos).

Datos:
{ultimos_datos}
"""

# Enviamos a Gemini
respuesta = modelo.generate_content(prompt)

print("📊 Análisis de Gemini:")
print("-" * 50)
print(respuesta.text)


### 🎯 Tu turno: Modifica el prompt

Experimenta cambiando el prompt para obtener diferentes análisis:


---

# 🏆 RETO BONUS: Dashboard Económico con IA

Construye tu propio **"Estado de la Economía Mexicana"** que integre todo lo que aprendiste.

El objetivo es generar un reporte automático que:
1. Obtenga datos de **Banxico** (tipo de cambio)
2. Obtenga datos de **INEGI** (IGAE)
3. Obtenga datos de **yfinance** (IPC de la bolsa)
4. Use **Gemini** para generar un análisis

**Output esperado:**
```
══════════════════════════════════════════
   ESTADO DE LA ECONOMÍA MEXICANA
══════════════════════════════════════════
📅 Fecha: [fecha actual]

💵 Tipo de cambio: $XX.XX MXN/USD
📊 IGAE: XXX.X puntos
📈 IPC Bolsa: XX,XXX puntos

──────────────────────────────────────────
📝 ANÁLISIS IA:
[Gemini genera un análisis de 2-3 párrafos]
──────────────────────────────────────────
```

### 🚀 ¿Quieres llevarlo al siguiente nivel?

Si quieres convertir esto en una aplicación web interactiva, te recomiendo explorar:

| Herramienta | Qué es | Ideal para |
|-------------|--------|------------|
| [Plotly](https://plotly.com/python/) | Gráficas interactivas | Dashboards con gráficas dinámicas |
| [Streamlit](https://streamlit.io/) | Apps web en Python | Prototipos rápidos, compartir con equipo |
| [Gradio](https://gradio.app/) | Interfaces para modelos IA | Demos de modelos de ML/IA |

Con **Streamlit** podrías tener tu dashboard corriendo en la nube en menos de 1 hora 🔥


---

# 📝 Resumen de la Clase

| Concepto | Descripción |
|----------|-------------|
| **API** | Interfaz para comunicar sistemas |
| **Request** | Petición que enviamos |
| **Response** | Respuesta que recibimos |
| **SDK** | Librería que simplifica una API |
| **API Key** | Llave de acceso/autenticación |

## APIs que usamos hoy:

| API/SDK | Uso | Autenticación |
|---------|-----|---------------|
| INEGI | Datos económicos de México | Token |
| Banxico | Indicadores del Banco de México | Token |
| yfinance | Datos financieros | No requiere |
| Gemini | Inteligencia Artificial | API Key gratuita |

---

### 📚 Recursos adicionales

- [Documentación INEGI](https://www.inegi.org.mx/servicios/api_indicadores.html)
- [yfinance en PyPI](https://pypi.org/project/yfinance/)
- [Google AI Studio](https://aistudio.google.com/)
- [APIs para economistas](https://github.com/milioe/APIs-for-economists)

---

**¿Dudas?** ✉️ emilio@milioe.com
